In [24]:
import cv2
import numpy as np
import math

def gaussian1D(m, s):
    r = np.random.normal(m, s, 1)  # s: standard deviation
    return r

def gaussian2D(m, s):
    r1 = np.random.normal(m, s, 1)  # s: standard deviation
    r2 = np.random.normal(m, s, 1)  # s: standard deviation
    return np.array([r1[0], r2[0]])

def likelihood(pixel_color, sigma):
    d = math.sqrt( (pixel_color[2]-255)**2 + (pixel_color[1]**2) + (pixel_color[0]**2) )
    return math.sqrt(1 / (2 * math.pi * sigma**2)) * math.exp(-(d**2) / (2 * sigma**2))

def display(numParticles, particles, frame, color):
    for i in range(numParticles):
        x = int(particles[0, i])
        y = int(particles[1, i])
        cv2.circle(frame, (x, y), 2, color, -1)
    cv2.imshow("Frame", frame)
    cv2.waitKey(1)


In [25]:
cap = cv2.VideoCapture('person.wmv')

imgH = 480
imgW = 640

In [26]:
targetColor = np.array([0, 0, 255])  # red
sigma = 80
posNoise = 15
velNoise = 5

numParticles = 5500

particles = np.array([
    np.random.randint(0, imgW, numParticles),
    np.random.randint(0, imgH, numParticles),
    3 * np.random.randn(numParticles) + 3,
    3 * np.random.randn(numParticles)
])

In [27]:
particles.shape

(4, 5500)

In [28]:
weights = np.ones(numParticles) / numParticles
predMat = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]])


In [29]:
if not cap.isOpened():
    print("Error opening video stream or file")

In [30]:
frameCount = 0
while cap.isOpened():
    ret, frame = cap.read()
    rawFrame = np.copy(frame)
    if ret:
        frameCount += 1
        if frameCount < 30:
            continue

        for i in range(numParticles):
            x = int(particles[0, i])
            y = int(particles[1, i])
            if x >= 0 and x < imgW and y >= 0 and y < imgH:
                pixel_color = frame[y, x]
                color_likelihood = likelihood(pixel_color,sigma)
                weights[i] = color_likelihood

        weights /= np.sum(weights)

        indices = np.random.choice(np.arange(numParticles), size=numParticles, replace=True, p=weights)
        particles = particles[:, indices]

        weights = np.ones(numParticles) / numParticles

        display(numParticles, particles, frame, (255, 0, 0))

        for i in range(numParticles):
            particles[:2, i] = np.dot(predMat[:2, :2], particles[:2, i]) + gaussian2D(0, posNoise)
            particles[2:, i] = np.dot(predMat[2:, 2:], particles[2:, i]) + gaussian2D(0, velNoise)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    else:
        break

cap.release()
cv2.destroyAllWindows()